In [3]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import Siddharth_Sanghavi_EDA.ipynb

ModuleNotFoundError: No module named 'Siddharth_Sanghavi_EDA'

In [ ]:
# Feature Engineering
customer_features = merged_data.groupby("CustomerID").agg({
    'TotalValue': 'sum',
    'Quantity': 'sum',
    'Price': 'mean'
}).reset_index()